In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numba import njit, prange

# 参数设置
num_particles = 100000
domain_size = 10.0  # cm
disc_radius = 3.0  # cm
temperature = 1000  # K
mass_electron = 9.10938356e-31  # kg
mass_ion = 1.6726219e-27  # kg (质子质量)
charge_electron = -1.60217662e-19  # C
charge_ion = 1.60217662e-19  # C
k_B = 1.38064852e-23  # J/K
num_steps = 1000
dt = 1e-10  # s (缩短时间步长)
epsilon_0 = 8.854187817e-12  # F/m (真空介电常数)

# 等效质量
effective_mass_electron = mass_electron * 1e-5
effective_mass_ion = mass_ion * 1e-5

# 计算粒子密度（m^-3）
volume = (2 * disc_radius / 100)**2  # 转换为m²
n = num_particles / volume  # 粒子密度

# 计算德拜长度
debye_length = np.sqrt(epsilon_0 * k_B * temperature / (n * charge_electron**2))

# 转换德拜长度到网格单元
grid_size = 60
cell_size = domain_size / grid_size  # cm
debye_radius_grid = int(np.ceil(debye_length / (cell_size / 100)))  # 转换为cm，再转换为网格单元

# 初始化粒子位置和速度
def initialize_positions_velocities(num_particles, disc_radius, mass, temperature):
    positions = np.random.uniform(-disc_radius, disc_radius, (num_particles, 2))
    velocities = np.random.normal(0, np.sqrt(k_B * temperature / mass), (num_particles, 2))
    distances = np.sqrt(np.sum(positions**2, axis=1))
    inside_disc = distances <= disc_radius
    return positions[inside_disc], velocities[inside_disc]

# 初始化电子和离子
electron_positions, electron_velocities = initialize_positions_velocities(num_particles, disc_radius, mass_electron, temperature)
ion_positions, ion_velocities = initialize_positions_velocities(num_particles, disc_radius, mass_ion, temperature)
num_particles_e = len(electron_positions)
num_particles_i = len(ion_positions)
charge_electron_array = np.full(num_particles_e, charge_electron)
charge_ion_array = np.full(num_particles_i, charge_ion)
mass_electron_array = np.full(num_particles_e, effective_mass_electron)
mass_ion_array = np.full(num_particles_i, effective_mass_ion)

# 电势和电场初始化
phi = np.zeros((grid_size, grid_size))
electric_field = np.zeros((grid_size, grid_size, 2))

# 泊松方程求解
@njit
def solve_poisson(phi, rho, num_iterations=100):
    for _ in range(num_iterations):
        phi_new = np.copy(phi)
        phi_new[1:-1, 1:-1] = (
            phi[2:, 1:-1] + phi[:-2, 1:-1] +
            phi[1:-1, 2:] + phi[1:-1, :-2] -
            rho[1:-1, 1:-1] * cell_size**2
        ) / 4.0
        phi = phi_new
    return phi

# 计算电场
@njit
def compute_electric_field(phi, cell_size):
    ex = (phi[2:, 1:-1] - phi[:-2, 1:-1]) / (2 * cell_size)
    ey = (phi[1:-1, 2:] - phi[1:-1, :-2]) / (2 * cell_size)
    return ex, ey

# 计算电荷密度
def compute_charge_density(positions, charge, grid_size, domain_size, cell_size):
    rho = np.zeros((grid_size, grid_size))
    bin_edges = [np.linspace(-domain_size / 2, domain_size / 2, grid_size + 1)] * 2
    hist, _, _ = np.histogram2d(positions[:, 0], positions[:, 1], bins=bin_edges, range=[[-domain_size / 2, domain_size / 2], [-domain_size / 2, domain_size / 2]])
    rho = hist * charge
    return rho

# 计算电场加速度
def compute_acceleration(positions, electric_field, domain_size, cell_size, charge, mass):
    grid_positions = np.floor((positions + domain_size / 2) / cell_size).astype(int)
    grid_positions = np.clip(grid_positions, 0, grid_size - 1)
    accelerations = np.zeros_like(positions)
    for i in range(len(positions)):
        ex = electric_field[grid_positions[i, 0], grid_positions[i, 1], 0]
        ey = electric_field[grid_positions[i, 0], grid_positions[i, 1], 1]
        accelerations[i] = (charge / mass) * np.array([ex, ey])
    return accelerations

# 更新粒子位置和速度（使用Verlet方法）
def verlet_update(positions, velocities, accelerations, new_accelerations, dt, domain_size):
    new_positions = positions + velocities * dt + 0.5 * accelerations * dt**2
    new_velocities = velocities + 0.5 * (accelerations + new_accelerations) * dt

    # 确保粒子不会离开方格边界
    for i in range(len(new_positions)):
        for j in range(2):
            if new_positions[i, j] < -domain_size / 2 or new_positions[i, j] > domain_size / 2:
                new_velocities[i, j] *= -1  # 反弹速度
                new_positions[i, j] = positions[i, j]  # 保持位置不变

    return new_positions, new_velocities

# 初始化电荷密度
rho_e = compute_charge_density(electron_positions, charge_electron, grid_size, domain_size, cell_size)
rho_i = compute_charge_density(ion_positions, charge_ion, grid_size, domain_size, cell_size)
rho = rho_e + rho_i

# 求解初始电势
phi = solve_poisson(phi, rho)

# 计算初始电场
ex, ey = compute_electric_field(phi, cell_size)
ex = np.pad(ex, ((1, 1), (1, 1)), mode='constant', constant_values=0)
ey = np.pad(ey, ((1, 1), (1, 1)), mode='constant', constant_values=0)
electric_field[:, :, 0] = ex
electric_field[:, :, 1] = ey

# 用于存储中线电子密度
midline_electron_density = []

In [ ]:

# 模拟
for step in range(num_steps + 1):  # 包括初始状态
    if step > 0:
        # 计算新的电荷密度
        rho_e = compute_charge_density(electron_positions, charge_electron, grid_size, domain_size, cell_size)
        rho_i = compute_charge_density(ion_positions, charge_ion, grid_size, domain_size, cell_size)
        rho = rho_e + rho_i

        # 解决泊松方程
        phi = solve_poisson(phi, rho)

        # 计算电场
        ex, ey = compute_electric_field(phi, cell_size)
        ex = np.pad(ex, ((1, 1), (1, 1)), mode='constant', constant_values=0)
        ey = np.pad(ey, ((1, 1)), mode='constant', constant_values=0)
        electric_field[:, :, 0] = ex
        electric_field[:, :, 1] = ey

        # 计算粒子加速度
        electron_acceleration = compute_acceleration(electron_positions, electric_field, domain_size, cell_size, charge_electron, effective_mass_electron)
        ion_acceleration = compute_acceleration(ion_positions, electric_field, domain_size, cell_size, charge_ion, effective_mass_ion)

        # 更新粒子位置和速度
        electron_positions, electron_velocities = verlet_update(electron_positions, electron_velocities, electron_acceleration, electron_acceleration, dt, domain_size)
        ion_positions, ion_velocities = verlet_update(ion_positions, ion_velocities, ion_acceleration, ion_acceleration, dt, domain_size)

    # 记录中线电子密度
    mid_index = grid_size // 2
    midline_electron_density.append(rho_e[mid_index, :])

    # 可视化
    if step % 10 == 0:
        fig, axs = plt.subplots(1, 3, figsize=(15, 5))
        axs[0].cla()
        axs[1].cla()
        axs[2].cla()

        hist_e, xedges, yedges = np.histogram2d(electron_positions[:, 0], electron_positions[:, 1], bins=grid_size, range=[[-domain_size / 2, domain_size / 2], [-domain_size / 2, domain_size / 2]])
        hist_i, _, _ = np.histogram2d(ion_positions[:, 0], ion_positions[:, 1], bins=grid_size, range=[[-domain_size / 2, domain_size / 2], [-domain_size / 2, domain_size / 2]])

        im1 = axs[0].imshow(hist_i.T, origin='lower', extent=[-domain_size / 2, domain_size / 2, -domain_size / 2, domain_size / 2], cmap='Reds', vmin=0, vmax=np.max(hist_i))
        axs[0].set_title('Ion Distribution')

        im2 = axs[1].imshow(hist_e.T, origin='lower', extent=[-domain_size / 2, domain_size / 2, -domain_size / 2, domain_size / 2], cmap='Blues', vmin=0, vmax=np.max(hist_e))
        axs[1].set_title('Electron Distribution')

        e_magnitude = np.sqrt(ex**2 + ey**2)
        im3 = axs[2].imshow(e_magnitude.T, origin='lower', extent=[-domain_size / 2, domain_size / 2, -domain_size / 2, domain_size / 2], cmap='plasma', vmin=0, vmax=np.max(e_magnitude))
        axs[2].set_title('Electric Field Strength (V/m)')

        fig.colorbar(im1, ax=axs[0], orientation='vertical', label='Number of Ions')
        fig.colorbar(im2, ax=axs[1], orientation='vertical', label='Number of Electrons')
        fig.colorbar(im3, ax=axs[2], orientation='vertical', label='Electric Field Strength (V/m)')

        plt.tight_layout()
        plt.savefig(f'step_{step:04d}.png')  # 保存图像
        plt.clf()  # 清除当前图像
        plt.close(fig)

print('所有图像已保存')

# 绘制电子密度沿中线的函数图
midline_density = np.array(midline_electron_density)
x_positions = np.linspace(-domain_size / 2, domain_size / 2, grid_size)

plt.figure(figsize=(10, 6))
for t in range(0, num_steps + 1, 100):  # 每100步绘制一次
    plt.plot(x_positions, midline_density[t], label=f'Step {t}')

plt.xlabel('Position along midline (cm)')
plt.ylabel('Electron Density')
plt.legend()
plt.title('Electron Density along Midline over Time')
plt.grid()
plt.show()


In [3]:
import numpy as np
import matplotlib.pyplot as plt
from numba import njit, prange

# 参数设置
num_particles = 100000
domain_size = 10.0  # cm
disc_radius = 3.0  # cm
temperature = 1000  # K
mass_electron = 9.10938356e-31  # kg
mass_ion = 1.6726219e-27  # kg (proton mass)
charge_electron = -1.60217662e-19  # C
charge_ion = 1.60217662e-19  # C
k_B = 1.38064852e-23  # J/K
num_steps = 1000
dt = 1e-10  # s (time step)
epsilon_0 = 8.854187817e-12  # F/m (vacuum permittivity)

# 等效质量
effective_mass_electron = mass_electron * 1e-5
effective_mass_ion = mass_ion * 1e-5

# 计算粒子密度（m^-3）
volume = (2 * disc_radius / 100)**2  # Convert to m²
n = num_particles / volume  # Particle density

# 计算德拜长度
debye_length = np.sqrt(epsilon_0 * k_B * temperature / (n * charge_electron**2))

# 转换德拜长度到网格单元
grid_size = 60
cell_size = domain_size / grid_size  # cm
debye_radius_grid = int(np.ceil(debye_length / (cell_size / 100)))  # Convert to cm, then to grid units

# 初始化粒子位置和速度
def initialize_positions_velocities(num_particles, disc_radius, mass, temperature):
    positions = np.random.uniform(-disc_radius, disc_radius, (num_particles, 2))
    velocities = np.random.normal(0, np.sqrt(k_B * temperature / mass), (num_particles, 2))
    distances = np.sqrt(np.sum(positions**2, axis=1))
    inside_disc = distances <= disc_radius
    return positions[inside_disc], velocities[inside_disc]

# 初始化电子和离子
electron_positions, electron_velocities = initialize_positions_velocities(num_particles, disc_radius, mass_electron, temperature)
ion_positions, ion_velocities = initialize_positions_velocities(num_particles, disc_radius, mass_ion, temperature)
num_particles_e = len(electron_positions)
num_particles_i = len(ion_positions)
charge_electron_array = np.full(num_particles_e, charge_electron)
charge_ion_array = np.full(num_particles_i, charge_ion)
mass_electron_array = np.full(num_particles_e, effective_mass_electron)
mass_ion_array = np.full(num_particles_i, effective_mass_ion)

# 电势和电场初始化
phi = np.zeros((grid_size, grid_size))
electric_field = np.zeros((grid_size, grid_size, 2))

# 泊松方程求解
@njit
def solve_poisson(phi, rho, num_iterations=100):
    for _ in range(num_iterations):
        phi_new = np.copy(phi)
        phi_new[1:-1, 1:-1] = (
            phi[2:, 1:-1] + phi[:-2, 1:-1] +
            phi[1:-1, 2:] + phi[1:-1, :-2] -
            rho[1:-1, 1:-1] * cell_size**2
        ) / 4.0
        phi = phi_new
    return phi

# 计算电场
@njit
def compute_electric_field(phi, cell_size):
    ex = (phi[2:, 1:-1] - phi[:-2, 1:-1]) / (2 * cell_size)
    ey = (phi[1:-1, 2:] - phi[1:-1, :-2]) / (2 * cell_size)
    return ex, ey

# 计算电荷密度
def compute_charge_density(positions, charge, grid_size, domain_size, cell_size):
    rho = np.zeros((grid_size, grid_size))
    bin_edges = [np.linspace(-domain_size / 2, domain_size / 2, grid_size + 1)] * 2
    hist, _, _ = np.histogram2d(positions[:, 0], positions[:, 1], bins=bin_edges, range=[[-domain_size / 2, domain_size / 2], [-domain_size / 2, domain_size / 2]])
    rho = hist * charge
    return rho

# 计算电场加速度
def compute_acceleration(positions, electric_field, domain_size, cell_size, charge, mass):
    grid_positions = np.floor((positions + domain_size / 2) / cell_size).astype(int)
    grid_positions = np.clip(grid_positions, 0, grid_size - 1)
    accelerations = np.zeros_like(positions)
    for i in range(len(positions)):
        ex = electric_field[grid_positions[i, 0], grid_positions[i, 1], 0]
        ey = electric_field[grid_positions[i, 0], grid_positions[i, 1], 1]
        accelerations[i] = (charge / mass) * np.array([ex, ey])
    return accelerations

# 更新粒子位置和速度（使用Verlet方法）
def verlet_update(positions, velocities, accelerations, new_accelerations, dt, domain_size):
    new_positions = positions + velocities * dt + 0.5 * accelerations * dt**2
    new_velocities = velocities + 0.5 * (accelerations + new_accelerations) * dt

    # 确保粒子不会离开方格边界
    for i in range(len(new_positions)):
        for j in range(2):
            if new_positions[i, j] < -domain_size / 2 or new_positions[i, j] > domain_size / 2:
                new_velocities[i, j] *= -1  # 反弹速度
                new_positions[i, j] = positions[i, j]  # 保持位置不变

    return new_positions, new_velocities

# 初始化电荷密度
rho_e = compute_charge_density(electron_positions, charge_electron, grid_size, domain_size, cell_size)
rho_i = compute_charge_density(ion_positions, charge_ion, grid_size, domain_size, cell_size)
rho = rho_e + rho_i

# 求解初始电势
phi = solve_poisson(phi, rho)

# 计算初始电场
ex, ey = compute_electric_field(phi, cell_size)
ex = np.pad(ex, ((1, 1), (1, 1)), mode='constant', constant_values=0)
ey = np.pad(ey, ((1, 1), (1, 1)), mode='constant', constant_values=0)
electric_field[:, :, 0] = ex
electric_field[:, :, 1] = ey

# 用于存储中线电子密度
midline_electron_density = []

# 模拟
for step in range(num_steps + 1):
    if step > 0:
        # 计算电子和离子之间的电场加速度
        electron_acceleration = compute_acceleration(electron_positions, electric_field, domain_size, cell_size, charge_electron_array, mass_electron_array)
        ion_acceleration = compute_acceleration(ion_positions, electric_field, domain_size, cell_size, charge_ion_array, mass_ion_array)

        # 更新电子位置和速度
        electron_positions, electron_velocities = verlet_update(electron_positions, electron_velocities, electron_acceleration, electron_acceleration, dt, domain_size)

        # 更新离子位置和速度
        ion_positions, ion_velocities = verlet_update(ion_positions, ion_velocities, ion_acceleration, ion_acceleration, dt, domain_size)

        # 计算新的电荷密度
        rho_e = compute_charge_density(electron_positions, charge_electron, grid_size, domain_size, cell_size)
        rho_i = compute_charge_density(ion_positions, charge_ion, grid_size, domain_size, cell_size)
        rho = rho_e + rho_i

        # 求解泊松方程
        phi = solve_poisson(phi, rho)

        # 计算电场
        ex, ey = compute_electric_field(phi, cell_size)
        ex = np.pad(ex, ((1, 1), (1, 1)), mode='constant', constant_values=0)
        ey = np.pad(ey, ((1, 1), (1, 1)), mode='constant', constant_values=0)
        electric_field[:, :, 0] = ex
        electric_field[:, :, 1] = ey

    # 可视化
    if step % 1 == 0:
        fig1, ax1 = plt.subplots(figsize=(8, 6))
        hist_i, xedges, yedges = np.histogram2d(ion_positions[:, 0], ion_positions[:, 1], bins=grid_size, range=[[-domain_size / 2, domain_size / 2], [-domain_size / 2, domain_size / 2]])
        im1 = ax1.imshow(hist_i.T, origin='lower', extent=[-domain_size / 2, domain_size / 2, -domain_size / 2, domain_size / 2], cmap='viridis')
        ax1.set_title(f'Ion Positions at Step {step}')
        ax1.set_xlabel('X (cm)')
        ax1.set_ylabel('Y (cm)')
        fig1.colorbar(im1, ax=ax1)
        fig1.savefig(f'ion_positions_step_{step}.png')
        plt.close(fig1)

        fig2, ax2 = plt.subplots(figsize=(8, 6))
        hist_e, xedges, yedges = np.histogram2d(electron_positions[:, 0], electron_positions[:, 1], bins=grid_size, range=[[-domain_size / 2, domain_size / 2], [-domain_size / 2, domain_size / 2]])
        im2 = ax2.imshow(hist_e.T, origin='lower', extent=[-domain_size / 2, domain_size / 2, -domain_size / 2, domain_size / 2], cmap='viridis')
        ax2.set_title(f'Electron Positions at Step {step}')
        ax2.set_xlabel('X (cm)')
        ax2.set_ylabel('Y (cm)')
        fig2.colorbar(im2, ax=ax2)
        fig2.savefig(f'electron_positions_step_{step}.png')
        plt.close(fig2)

        fig3, ax3 = plt.subplots(figsize=(8, 6))
        ex_field = np.sqrt(electric_field[:, :, 0]**2 + electric_field[:, :, 1]**2)
        im3 = ax3.imshow(ex_field.T, origin='lower', extent=[-domain_size / 2, domain_size / 2, -domain_size / 2, domain_size / 2], cmap='plasma')
        ax3.set_title(f'Electric Field Magnitude at Step {step}')
        ax3.set_xlabel('X (cm)')
        ax3.set_ylabel('Y (cm)')
        fig3.colorbar(im3, ax=ax3, label='Electric Field Magnitude (V/m)')
        fig3.savefig(f'electric_field_step_{step}.png')
        plt.close(fig3)

        # 计算中线电子密度
        midline_hist, _ = np.histogram(electron_positions[:, 0], bins=grid_size, range=[-domain_size / 2, domain_size / 2])
        midline_electron_density.append(midline_hist)

# 生成电子密度的中线函数图
fig4, ax4 = plt.subplots(figsize=(8, 6))
midline_electron_density = np.array(midline_electron_density)
mean_density = np.mean(midline_electron_density, axis=0)
ax4.plot(np.linspace(-domain_size / 2, domain_size / 2, grid_size), mean_density, label='Electron Density on Midline')
ax4.set_title('Midline Electron Density Over Time')
ax4.set_xlabel('X (cm)')
ax4.set_ylabel('Electron Density (particles/cm)')
ax4.legend()
fig4.savefig('midline_electron_density.png')
plt.close(fig4)


C:\Users\0\AppData\Local\Temp\ipykernel_61140\3293719428.py:162: UserWarning: Glyph 31163 (\N{CJK UNIFIED IDEOGRAPH-79BB}) missing from current font.
  fig1.savefig(f'ion_positions_step_{step}.png')
C:\Users\0\AppData\Local\Temp\ipykernel_61140\3293719428.py:162: UserWarning: Glyph 23376 (\N{CJK UNIFIED IDEOGRAPH-5B50}) missing from current font.
  fig1.savefig(f'ion_positions_step_{step}.png')
C:\Users\0\AppData\Local\Temp\ipykernel_61140\3293719428.py:162: UserWarning: Glyph 20301 (\N{CJK UNIFIED IDEOGRAPH-4F4D}) missing from current font.
  fig1.savefig(f'ion_positions_step_{step}.png')
C:\Users\0\AppData\Local\Temp\ipykernel_61140\3293719428.py:162: UserWarning: Glyph 32622 (\N{CJK UNIFIED IDEOGRAPH-7F6E}) missing from current font.
  fig1.savefig(f'ion_positions_step_{step}.png')
C:\Users\0\AppData\Local\Temp\ipykernel_61140\3293719428.py:169: UserWarning: Glyph 30005 (\N{CJK UNIFIED IDEOGRAPH-7535}) missing from current font.
  fig2.savefig(f'electron_positions_step_{step}.png')


KeyboardInterrupt: 